# Import

In [1]:
import os
from glob import glob
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.utils import shuffle
from keras.utils import CustomObjectScope 
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score

import tensorflow as tf
from keras import backend as K
from keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from keras.models import Model
from keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from keras.optimizers import Adam
from keras.metrics import Recall, Precision


# Data path


In [2]:
project_path = r"D:\Liver160"

# Utills


In [3]:
def create_dir(path):
    path = os.path.join(project_path, path)
    if not os.path.exists(path):
        os.makedirs(path)


In [4]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

In [5]:
smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)


In [6]:
def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [7]:
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x


In [8]:
def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p


def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x


In [9]:
H = 160
W = 160
def save_results(image, mask, y_pred, save_image_path):
    ## i - m - y
    line = np.ones((H, 10, 3)) * 128

    """ Mask """
    mask = np.expand_dims(mask, axis=-1)    ## (512, 512, 1)
    mask = np.concatenate([mask, mask, mask], axis=-1)  ## (512, 512, 3)

    """ Predicted Mask """
    y_pred = np.expand_dims(y_pred, axis=-1)    ## (512, 512, 1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)  ## (512, 512, 3)
    y_pred = y_pred * 255

    cat_images = np.concatenate([image, line, mask, line, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)


# Build model

In [10]:
H = 160
W = 160

In [11]:
def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 20)
    s2, p2 = encoder_block(p1, 40)
    s3, p3 = encoder_block(p2, 80)
    s4, p4 = encoder_block(p3, 160)

    b1 = conv_block(p4, 320)

    d1 = decoder_block(b1, s4, 160)
    d2 = decoder_block(d1, s3, 80)
    d3 = decoder_block(d2, s2, 40)
    d4 = decoder_block(d3, s1, 20)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="U-Net")
    return model

input_shape = (160, 160, 3)
model = build_unet(input_shape)
model.summary()

Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 160, 160, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 160, 160, 20  560         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 160, 160, 20  80         ['conv2d[0][0]']                 
 alization)                     )                                                             

# Train

In [12]:
def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.jpg")))
    y = sorted(glob(os.path.join(path, "mask", "*.jpg")))
    return x, y

In [13]:
def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

In [14]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = x/255.0
    x = x.astype(np.float32)
    return x


In [15]:
def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = x/255.0
    x = x > 0.5
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x


In [16]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y


In [17]:
def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset


In [18]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Directory for storing files """
create_dir("files")
model_path = os.path.join(project_path, "files", "model.h5")
csv_path = os.path.join(project_path, "files", "data.csv")


""" Hyperparameters """
batch_size = 2
lr = 1e-4
num_epochs = 5

""" Dataset """
dataset_path = os.path.join(project_path, "new_data")
train_path = os.path.join(dataset_path, "train")
valid_path = os.path.join(dataset_path, "valid")

train_x, train_y = load_data(train_path)
train_x, train_y = shuffling(train_x, train_y)
valid_x, valid_y = load_data(valid_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")

train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

""" Model """
model = build_unet((H, W, 3))
metrics = [dice_coef, iou, Recall(), Precision()]
model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=metrics)

callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=1e-7, verbose=1),
    CSVLogger(csv_path),
    TensorBoard(),
    EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=False),
    ]

model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    callbacks=callbacks,
    shuffle=False
    )


Train: 20512 - 20512
Valid: 1281 - 1281
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Epoch 1/5
10256/10256 [==============================] - ETA: 0s - loss: 0.2150 - dice_coef: 0.7850 - iou: 0.6900 - recall: 0.9163 - precision: 0.7868
Epoch 1: val_loss improved from inf to 0.13925, saving model to D:\Liver160\files\model.h5
10256/10256 [==============================] - 3322s 324ms/step - loss: 0.2150 - dice_coef: 0.7850 - iou: 0.6900 - recall: 0.9163 - precision: 0.7868 - val_loss: 0.1393 - val_dice_coef: 0.8607 - val_iou: 0.7973 - val_recall: 0.9344 - val_precision: 0.9209 - lr: 1.0000e-04
Epoch 2/5
10256/10256 [==============================] - ETA: 0s - loss: 0.0750 - dice_coef: 0.9250 - iou: 0.8680 - recall: 0.9411 - precision: 0.9375
Epoch 2: val_loss improved from 0.13925 to 0.10094, saving model to D:\Liver160\files\model.h

KeyboardInterrupt: 

# Evalution

In [19]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Directory for storing files """
create_dir("results")

""" Loading model """
with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
    model = tf.keras.models.load_model(model_path)

""" Load the dataset """
test_x = sorted(glob(os.path.join(project_path, "new_data", "valid", "image", "*")))
test_y = sorted(glob(os.path.join(project_path, "new_data", "valid", "mask", "*")))
print(f"Test: {len(test_x)} - {len(test_y)}")

""" Evaluation and Prediction """
results = os.path.join(project_path, "results")
SCORE = []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extract the name """
    name = x.split(os.sep)[-1].split(".")[0]

    """ Reading the image """
    image = cv2.imread(x, cv2.IMREAD_COLOR)
    x = image/255.0
    x = np.expand_dims(x, axis=0)

    """ Reading the mask """
    mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    y = mask/255.0
    y = y > 0.5
    y = y.astype(np.int32)

    """ Prediction """
    y_pred = model.predict(x)[0]
    y_pred = np.squeeze(y_pred, axis=-1)
    y_pred = y_pred > 0.5
    y_pred = y_pred.astype(np.int32)
        
    """ Saving the prediction """
    
    save_image_path = results + f"/{name}.png"
    save_results(image, mask, y_pred, save_image_path)

    """ Flatten the array """
    y = y.flatten()
    y_pred = y_pred.flatten()
    """ Calculating the metrics values """
    acc_value = accuracy_score(y, y_pred)
    f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary", zero_division=1)
    jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary", zero_division=1)
    recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary", zero_division=1)
    precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary", zero_division=1)
    SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])

""" Metrics values """
score = [s[1:]for s in SCORE]
score = np.mean(score, axis=0)
print(f"Accuracy: {score[0]:0.5f}")
print(f"F1: {score[1]:0.5f}")
print(f"Jaccard: {score[2]:0.5f}")
print(f"Recall: {score[3]:0.5f}")
print(f"Precision: {score[4]:0.5f}")

df = pd.DataFrame(SCORE, columns=["Image", "Accuracy", "F1", "Jaccard", "Recall", "Precision"])
df.to_csv(csv_path)


Test: 1281 - 1281


  0%|          | 0/1281 [00:00<?, ?it/s]

1/1 [==============================] - 0s 389ms/step


  0%|          | 1/1281 [00:00<09:50,  2.17it/s]

1/1 [==============================] - 0s 59ms/step


...

1/1 [==============================] - 0s 57ms/step


100%|█████████▉| 1280/1281 [02:19<00:00,  9.29it/s]

1/1 [==============================] - 0s 55ms/step


100%|██████████| 1281/1281 [02:19<00:00,  9.18it/s]

Accuracy: 0.99193
F1: 0.89255
Jaccard: 0.83788
Recall: 0.93835
Precision: 0.88860
